# Instalação e configuração do ambiente

Esta célula instala as dependências necessárias para a execução do experimento. Inclui a configuração do Java 22, essencial para o funcionamento das ferramentas CK e Designite, além da instalação do Maven e das bibliotecas Python para análise e aprendizado de máquina.

É necessária a substituição do OpenJDK 17 para a versão 22, para o funcionamento da ferramente Designite.



In [ ]:
pip install PyGithub


In [ ]:
!wget https://github.com/adoptium/temurin22-binaries/releases/download/jdk-22.0.2%2B9/OpenJDK22U-jdk_x64_linux_hotspot_22.0.2_9.tar.gz
!tar -xzf OpenJDK22U-jdk_x64_linux_hotspot_22.0.2_9.tar.gz
!mv jdk-22.0.2+9 /usr/local/jdk-22


In [ ]:
!sudo update-alternatives --install /usr/bin/java java /usr/local/jdk-22/bin/java 1
!sudo update-alternatives --set java /usr/local/jdk-22/bin/java


In [ ]:
!export JAVA_HOME=/usr/local/jdk-22/bin/java
!export PATH=$JAVA_HOME/bin:$PATH


In [ ]:
!git clone https://github.com/mauricioaniche/ck.git /content/ck

In [ ]:
%cd /content/ck
!./mvnw clean package -Dmaven.javadoc.skip=true -DskipTests


In [ ]:
!mkdir -p /content/designite

!wget https://www.designite-tools.com/assets/DesigniteJava.jar -O DesigniteJava.jar /content/designite


In [ ]:
!mv /content/ck/DesigniteJava.jar /content/designite/
!ls -lh /content/designite/

# Clonagem e preparação dos repositórios

Nesta etapa, são clonados projetos Java open source que servirão de base para a análise, incluindo java-design-patterns, EvoSuite, SonarQube. Esses repositórios representam diferentes estilos de código e padrões arquiteturais, oferecendo diversidade para o conjunto de dados.

In [ ]:
!git clone https://github.com/iluwatar/java-design-patterns.git repos_java/java-design-patterns
!git clone https://github.com/EvoSuite/evosuite.git repos_java/evosuite
!git clone https://github.com/SonarSource/sonarqube.git repos_java/sonarqube


# Execução da ferramenta Designite

A ferramenta DesigniteJava é usada para detectar automaticamente code smells de diferentes categorias: Design Smells, Implementation Smells, Architecture Smells e Test Smells.

Cada análise gera um conjunto de arquivos .csv por projeto, listando classes e tipos de smells detectados.
Em seguida, todos os arquivos são consolidados em um dataset unificado (designite_dataset.csv).

In [ ]:
import subprocess
import glob
import os

designite_path = "/content/designite/DesigniteJava.jar"
base_path = "/content/ck/repos_java"

java_paths = glob.glob(f"{base_path}/**/src/main/java", recursive=True)

print(f"{len(java_paths)} projetos detectados com código-fonte Java.\n")

for path in java_paths:
    project_dir = os.path.abspath(os.path.join(path, "../../.."))
    output_dir = os.path.join(project_dir, "designite_output")
    os.makedirs(output_dir, exist_ok=True)

    print(f"Analisando projeto: {project_dir}")
    try:
        subprocess.run(
            ["java", "-jar", designite_path, "-i", project_dir, "-o", output_dir],
            check=True
        )
        print(f"Análise concluída: {output_dir}\n")
        type_metrics_file = os.path.join(output_dir, "TypeMetrics.csv")
        if os.path.exists(type_metrics_file):
            with open(type_metrics_file, "r") as f:
                class_count = sum(1 for _ in f) - 1
            print(f"→ Classes analisadas pelo Designite: {class_count}")

        java_files = glob.glob(f"{project_dir}/**/*.java", recursive=True)
        print(f"→ Arquivos Java encontrados: {len(java_files)}\n")

    except subprocess.CalledProcessError as e:
        print(f"Erro ao analisar {project_dir}: {e}\n")


In [ ]:
import subprocess, os, glob

designite_path = "/content/designite/DesigniteJava.jar"

for repo in glob.glob("/content/ck/repos_java/*"):
    out_dir = os.path.join(repo, "designite_output")
    os.makedirs(out_dir, exist_ok=True)
    cmd = ["java", "-jar", designite_path, "-i", repo, "-o", out_dir]
    print(f"Analisando {repo} ...")
    subprocess.run(cmd)


In [ ]:
import pandas as pd
import glob

files = glob.glob("/content/ck/repos_java/*/designite_output/DesignSmells.csv")
df_list = [pd.read_csv(f) for f in files]
df_smells = pd.concat(df_list, ignore_index=True)

df_smells.to_csv("/content/dataset_code_smells.csv", index=False)
print("Dataset rotulado salvo como dataset_code_smells.csv")


In [ ]:
import pandas as pd
import glob

# caminhos dos resultados
paths = glob.glob("/content/ck/repos_java/**/designite_output/*.csv", recursive=True)

# lista para armazenar todos os DataFrames
frames = []

for path in paths:
    category = path.split("/")[-1].replace(".csv", "")
    df = pd.read_csv(path)
    df["Category"] = category  # adiciona a categoria (Design, Implementation, etc)
    frames.append(df)

dataset = pd.concat(frames, ignore_index=True)
dataset.to_csv("/content/designite_dataset.csv", index=False)

print(f"Dataset consolidado com {len(dataset)} registros salvo em /content/designite_dataset.csv")
print("Categorias encontradas:", dataset['Category'].unique())


# Execução da ferramenta CK

A ferramenta CK Metrics é executada em todas as pastas que contêm código-fonte Java (src/main/java).
Ela gera arquivos CSV contendo métricas estruturais de classes e métodos, como:


*   WMC (Complexidade Ciclomática)
*   CBO (Acoplamento)
*   LCOM (Coesão)
*   RFC (Resposta por Classe)
*   LOC (Linhas de Código)

Esses dados formam a base quantitativa para a detecção automática de code smells.

In [ ]:
import subprocess
import os
import glob

ck_path = "/content/ck/target/ck-0.7.1-SNAPSHOT-jar-with-dependencies.jar"

# pasta onde todos os repositórios estão clonados
repos_root = "/content/ck/repos_java"

# percorrer todas as pastas dos projetos que contenham 'src/main/java'
java_paths = glob.glob(f"{repos_root}/**/src/main/java", recursive=True)

print(f"Total de pastas Java encontradas: {len(java_paths)}\n")

for path in java_paths:
    print(f"Analisando pasta: {path}")
    output_dir = os.path.join(path, "ck_output")

    os.makedirs(output_dir, exist_ok=True)

    subprocess.run([
        "java", "-jar", ck_path,
        path,
        "true", "0", "false",
        output_dir
    ])

# Geração de rótulos automáticos

Como nem todas as classes possuem anotações manuais de smells, é feita uma rotulação heurística.
Classes com valores acima da média para métricas como WMC, CBO, LCOM ou RFC são marcadas como code_smell = 1.

In [ ]:
import pandas as pd
import glob
import os

base_path = "/content/ck/repos_java"

# (procura qualquer arquivo dentro de pastas que contenham 'ck_output')
output_files = glob.glob(f"{base_path}/**/ck_output*.csv", recursive=True)
print(f"{len(output_files)} arquivos CSV encontrados para consolidação.\n")

if not output_files:
    raise FileNotFoundError("Nenhum arquivo de métricas CK encontrado nas pastas de saída.")

frames = []

for f in output_files:
    try:
        df_part = pd.read_csv(f)
        df_part["source_file"] = os.path.basename(f)  # rastreabilidade
        df_part["project"] = f.split("/")[3] if len(f.split("/")) > 3 else "unknown"
        frames.append(df_part)
        print(f"Lido: {f} ({df_part.shape[0]} linhas)")
    except Exception as e:
        print(f"Erro ao ler {f}: {e}")

# Consolidar em um único DataFrame
if frames:
    df = pd.concat(frames, ignore_index=True)
    print(f"\n dataset combinado com {df.shape[0]} linhas e {df.shape[1]} colunas.")

    # criar coluna de rótulo
    if {"wmc", "cbo", "lcom", "rfc"}.issubset(df.columns):
        limiar_wmc = df["wmc"].mean()
        limiar_cbo = df["cbo"].mean()
        limiar_lcom = df["lcom"].mean()
        limiar_rfc = df["rfc"].mean()

        df["code_smell"] = (
            (df["wmc"] > limiar_wmc) |
            (df["cbo"] > limiar_cbo) |
            (df["lcom"] > limiar_lcom) |
            (df["rfc"] > limiar_rfc)
        ).astype(int)

        print(f"Rótulo 'code_smell' criado com base em múltiplas métricas (WMC, CBO, LCOM, RFC)")
    else:
        df["code_smell"] = 0
        print("Métricas principais ausentes. Criada coluna 'code_smell' dummy = 0.")

    output_path = "/content/dataset_metrics.csv"
    df.to_csv(output_path, index=False)
    print(f"\nDataset consolidado salvo em: {output_path}")
else:
    print("Nenhum arquivo CSV válido foi processado.")


In [ ]:
df['code_smell'].value_counts()

# Consolidação dos datasets

Os arquivos gerados pelas ferramentas CK e Designite são integrados, formando o dataset final dataset_final.csv.
A junção é realizada com base nos identificadores de arquivo e classe, combinando métricas estruturais (features) e rótulos de smells (labels).
Esse dataset é a principal entrada para os modelos de aprendizado de máquina.

In [ ]:
df_metrics = pd.read_csv("/content/dataset_metrics.csv")
df_smells = pd.read_csv("/content/designite_dataset.csv")

dataset_final = pd.merge(df_metrics, df_smells,
                         left_on=["file", "class"],
                         right_on=["File", "Class"],
                         how="left")

dataset_final.to_csv("/content/dataset_final.csv", index=False)
print("Dataset final salvo como dataset_final.csv")

# Balanceamento de classes (SMOTE)

O dataset costuma estar desbalanceado, com muito mais exemplos de classes "limpas" do que "com smell".
Por isso, é aplicado o SMOTE (Synthetic Minority Over-sampling Technique), que cria amostras sintéticas da classe minoritária para melhorar o equilíbrio dos dados e evitar viés nos modelos.

# Treinamento dos modelos de Machine Learning

São avaliados dois algoritmos supervisionados:

* Random Forest, com ajuste de parâmetros (n_estimators, max_depth, etc.) via GridSearchCV.

* SVM (Support Vector Machine), com ajuste de C e gamma.

As variáveis são normalizadas com StandardScaler, e a separação treino/teste é feita em 80/20, de forma estratificada.

# Avaliação e comparação de desempenho

Os modelos são avaliados com base nas métricas:

* Acurácia, Precisão, Revocação e F1-Score

* Matriz de Confusão para inspeção dos erros.

Os resultados são comparados em um gráfico e salvos em resultados_modelos.csv.
Essa etapa atende ao objetivo central de avaliar a eficácia dos algoritmos de aprendizado de máquina na classificação de code smells.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    precision_score, recall_score, f1_score, make_scorer
)
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

df = pd.read_csv("/content/dataset_final.csv")
print(f"Dataset carregado: {df.shape[0]} linhas e {df.shape[1]} colunas\n")

df_numeric = df.select_dtypes(include=["int64", "float64"]).dropna(axis=1, how="all").fillna(0)

if "code_smell" not in df_numeric.columns:
    raise ValueError("A coluna 'code_smell' não foi encontrada no dataset!")

X = df_numeric.drop(columns=["code_smell"])
y = df_numeric["code_smell"]

print(f"Variáveis de entrada: {X.shape[1]} | Classes: {y.value_counts().to_dict()}\n")

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
print("Validação cruzada configurada (10-Fold estratificada).\n")

rf_pipeline = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('model', RandomForestClassifier(random_state=42))
])

svm_pipeline = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('scaler', StandardScaler()),
    ('model', SVC(kernel='rbf', random_state=42))
])

rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

svm_params = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto']
}

scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, zero_division=0),
    'recall': make_scorer(recall_score, zero_division=0),
    'f1': make_scorer(f1_score, zero_division=0)
}

print("Ajustando hiperparâmetros com GridSearchCV\n")

rf_grid = GridSearchCV(rf_pipeline, rf_params, cv=3, scoring='accuracy', n_jobs=-1)
svm_grid = GridSearchCV(svm_pipeline, svm_params, cv=3, scoring='accuracy', n_jobs=-1)

rf_grid.fit(X, y)
svm_grid.fit(X, y)

print("GridSearch finalizado!")
print(f"Melhores parâmetros Random Forest: {rf_grid.best_params_}")
print(f"Melhores parâmetros SVM: {svm_grid.best_params_}\n")

def avaliar_modelo(modelo, nome):
    resultados = cross_validate(
        modelo, X, y,
        cv=kfold, scoring=scoring,
        n_jobs=-1, return_train_score=False
    )
    df_res = pd.DataFrame(resultados)
    mean = df_res.mean()
    std = df_res.std()
    print(f"\n{nome} — Resultados médios (10-Fold):")
    print(mean[['test_accuracy', 'test_precision', 'test_recall', 'test_f1']])
    return mean, std

rf_mean, rf_std = avaliar_modelo(rf_grid.best_estimator_, "Random Forest")
svm_mean, svm_std = avaliar_modelo(svm_grid.best_estimator_, "SVM")

comparativo = pd.DataFrame({
    'Modelo': ['Random Forest', 'SVM'],
    'Acurácia Média': [rf_mean['test_accuracy'], svm_mean['test_accuracy']],
    'Precisão Média': [rf_mean['test_precision'], svm_mean['test_precision']],
    'Recall Médio': [rf_mean['test_recall'], svm_mean['test_recall']],
    'F1-Score Médio': [rf_mean['test_f1'], svm_mean['test_f1']],
    'Desvio Padrão (Acc)': [rf_std['test_accuracy'], svm_std['test_accuracy']]
}).round(4)

print("\n=== Comparação Final (Média e Desvio-Padrão em 10-Fold) ===\n")
print(comparativo)


sns.set(style="whitegrid")
plt.figure(figsize=(7,4))
sns.barplot(x='Modelo', y='Acurácia Média', data=comparativo, palette='Blues_d')
plt.title("Comparação de Acurácia Média (10-Fold)")
plt.ylabel("Acurácia Média")
plt.xlabel("")
plt.tight_layout()
plt.show()


comparativo.to_csv("/content/resultados.csv", index=False)
print("\nResultados salvos em '/content/resultados.csv'")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE

df = pd.read_csv("/content/dataset_final.csv")
df = df.select_dtypes(include=["int64", "float64"]).dropna(axis=1, how="all").fillna(0)

X = df.drop(columns=["code_smell"])
y = df["code_smell"]

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42, stratify=y_res)

rf = RandomForestClassifier(n_estimators=200, max_depth=20, min_samples_split=2, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm = SVC(C=10, gamma="scale", kernel="rbf", random_state=42)
svm.fit(X_train_scaled, y_train)
svm_pred = svm.predict(X_test_scaled)

def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(title)
    plt.xlabel("Predito")
    plt.ylabel("Real")
    plt.show()

plot_confusion_matrix(y_test, rf_pred, "Matriz de Confusão — Random Forest")
plot_confusion_matrix(y_test, svm_pred, "Matriz de Confusão — SVM")

# Hipóteses

 H₀ (nula): não há diferença significativa entre os modelos (as médias das métricas são iguais).

 H₁ (alternativa): há diferença significativa entre os modelos.

# Aplicação do teste

* Como os dois modelos são comparados sobre as mesmas dobras de dados, usa-se um teste pareado, como:

  * Teste t de Student pareado, se os dados seguem distribuição normal;

  * Teste de Wilcoxon signed-rank, se não houver normalidade (não paramétrico).

In [ ]:
from scipy.stats import ttest_rel, wilcoxon

# valores de acurácia de cada modelo
acc_rf = [1.0,1.0,1.0,1.0,1.0,1.0,0.9998189717595944,0.9998189717595944,1.0,1.0]
acc_svm = [0.9780995475113122,0.9748416289592761,0.9777335264301231,0.9784576393917451,0.974475018102824,0.973388848660391,0.9764663287472846,0.9764663287472846,0.973388848660391,0.975199131064446]

# Teste t pareado
t_stat, p_val = ttest_rel(acc_rf, acc_svm)

print(f"t = {t_stat:.3f}, p = {p_val:.4f}")

if p_val < 0.05:
    print("Diferença estatisticamente significativa (rejeita H0).")
else:
    print("Sem diferença significativa (aceita H0).")
